# Houston, TX Simulations
Need to add Scenario field to CSV files.<br>
8760 rows per simulation<br>
49 files 

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [12]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

# Size (in sq ft) of the building to establish EUI
size = 2400

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

---

## Combine v9.5 CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/95/hou/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data95 = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/95/hou/" + file)
    all_data95 = pd.concat([all_data95, df])
    
all_data95

# 429240  rows, 214 columns

,Scenario,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Direct Solar Radiation Rate per Area [W/m2](Hourly),Environment:Site Exterior Horizontal Sky Illuminance [lux](Hourly),Environment:Site Exterior Horizontal Beam Illuminance [lux](Hourly),Environment:Site Exterior Beam Normal Illuminance [lux](Hourly),Environment:Site Sky Diffuse Solar Radiation Luminous Efficacy [lum/W](Hourly),Environment:Site Beam Solar Radiation Luminous Efficacy [lum/W](Hourly),...,WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 2 View Luminance [cd/m2](Hourly),LIVING_UNIT1:Zone Operative Temperature [C](Hourly),ATTIC_UNIT1:Zone Operative Temperature [C](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 90% Acceptability Status [](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model 80% Acceptability Status [](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature [C](Hourly),PEOPLE_UNIT1:Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature [C](Hourly),SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly)
0,1,01/01 01:00:00,2.9875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.650323,5.597444,1.0,1.0,12.300694,21.613215,481868.92660,1.465875e+07,0.0
1,1,01/01 02:00:00,2.8000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.558553,5.005730,1.0,1.0,12.300694,21.613215,529666.34510,1.611278e+07,0.0
2,1,01/01 03:00:00,2.4250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.496253,4.632116,1.0,1.0,12.300694,21.613215,571558.93460,1.738718e+07,0.0
3,1,01/01 04:00:00,1.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.452241,4.200931,1.0,1.0,12.300694,21.613215,611912.21050,1.861475e+07,0.0
4,1,01/01 05:00:00,1.7000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,20.424784,3.807051,1.0,1.0,12.300694,21.613215,631186.01780,1.920107e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,49,12/31 20:00:00,11.1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,23.526313,15.381476,1.0,1.0,12.613681,21.710241,0.00000,0.000000e+00,0.0
8756,49,12/31 21:00:00,10.0375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,22.731577,13.298693,1.0,1.0,12.613681,21.710241,0.00000,0.000000e+00,0.0
8757,49,12/31 22:00:00,8.0250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,21.784661,11.687463,1.0,1.0,12.613681,21.710241,24745.44308,7.527717e+05,0.0
8758,49,12/31 23:00:00,6.8875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,21.209938,10.089124,1.0,1.0,12.613681,21.710241,207743.95540,6.319700e+06,0.0


In [5]:
# Get wanted columns
all_data95_1 = all_data95[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

all_data95_1

,Scenario,Date/Time,LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly),SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly)
0,1,01/01 01:00:00,0.0,481868.92660,1.465875e+07,0.0
1,1,01/01 02:00:00,0.0,529666.34510,1.611278e+07,0.0
2,1,01/01 03:00:00,0.0,571558.93460,1.738718e+07,0.0
3,1,01/01 04:00:00,0.0,611912.21050,1.861475e+07,0.0
4,1,01/01 05:00:00,0.0,631186.01780,1.920107e+07,0.0
...,...,...,...,...,...,...
8755,49,12/31 20:00:00,0.0,0.00000,0.000000e+00,0.0
8756,49,12/31 21:00:00,0.0,0.00000,0.000000e+00,0.0
8757,49,12/31 22:00:00,0.0,24745.44308,7.527717e+05,0.0
8758,49,12/31 23:00:00,0.0,207743.95540,6.319700e+06,0.0


In [6]:
# all_data95_1 = all_data95_1.dropna().reset_index(drop=True)

# all_data95_1

# #438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [14]:
# Rename Columns
all_data95_2 = all_data95_1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data95_2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [15]:
# Copy dataframe to modify and leave original df intact
get_annual95 = all_data95_2.copy(deep=True)

#get_annual

In [16]:
# Convert Joules to KBtu in dataframe
get_annual95["FanEnergy(kBtu)"] = get_annual95["FanEnergy[J](Hourly)"] * .00000094781712
get_annual95["HeatingEnergy(kBtu)"] = get_annual95["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual95["CoolingEnergy(kBtu)"] = get_annual95["CoolingEnergy[J](Hourly) "] * .00000094781712

get_annual95

,Scenario,Date_Time,Illuminance(lux),FanEnergy[J](Hourly),HeatingEnergy[J](Hourly),CoolingEnergy[J](Hourly),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu)
0,1,01/01 01:00:00,0.0,481868.92660,1.465875e+07,0.0,0.456724,13.893815,0.0
1,1,01/01 02:00:00,0.0,529666.34510,1.611278e+07,0.0,0.502027,15.271967,0.0
2,1,01/01 03:00:00,0.0,571558.93460,1.738718e+07,0.0,0.541733,16.479864,0.0
3,1,01/01 04:00:00,0.0,611912.21050,1.861475e+07,0.0,0.579981,17.643377,0.0
4,1,01/01 05:00:00,0.0,631186.01780,1.920107e+07,0.0,0.598249,18.199102,0.0
...,...,...,...,...,...,...,...,...,...
8755,49,12/31 20:00:00,0.0,0.00000,0.000000e+00,0.0,0.000000,0.000000,0.0
8756,49,12/31 21:00:00,0.0,0.00000,0.000000e+00,0.0,0.000000,0.000000,0.0
8757,49,12/31 22:00:00,0.0,24745.44308,7.527717e+05,0.0,0.023454,0.713490,0.0
8758,49,12/31 23:00:00,0.0,207743.95540,6.319700e+06,0.0,0.196903,5.989920,0.0


In [10]:
#list(get_annual95.columns.values)

In [11]:
# Drop columns
get_annual95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual95

In [ ]:
# Get the source energy EUI (energy/sq ft)

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [ ]:
# Get annual costs
get_annual95_costs = all_data95_2.copy(deep=True)

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual95_costs["FanCost($)"] = (get_annual95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual95_costs["HeatingCost($)"] = ((get_annual95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual95_costs["CoolingCost($)"] = (get_annual95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs95

In [ ]:
# Drop columns
get_annual95_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs95

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [ ]:
# Combine data for energy and cost
allAnnualData95 = pd.merge(get_annual95, get_annual95_costs)

#allAnnualData95

In [ ]:
# Add energy and cost totals
totalEnergy95 = allAnnualData95["FanEnergy(kBtu)"] + allAnnualData95["HeatingEnergy(kBtu)"] + allAnnualData95["CoolingEnergy(kBtu)"]
allAnnualData95["AnnualEnergy(kBtu)"] = totalEnergy95

totalCost95 = allAnnualData95["FanCost($)"] + allAnnualData95["HeatingCost($)"] + allAnnualData95["CoolingCost($)"]
allAnnualData95["AnnualCost($)"] = totalCost95

#allAnnualData95

In [ ]:
#list(allAnnualData95.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [ ]:
# Group by scenario
annualData95 = allAnnualData95.groupby(["Scenario"], as_index=True)

#annualData95.sum().round(2)

In [ ]:
annualDataFinal95 = annualData95.sum().round(2)

#annualDataFinal95

In [ ]:
# Export grouped file to csv
annualDataFinal95.to_csv("../Regression_Scraper_Output/hou_annual95_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh for costs only, convert to kbtu for data<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [ ]:
# Copy dataframe to modify and leave original df intact
get_monthly95 = all_data95_2.copy(deep=True)

#get_monthly95

In [ ]:
# Convert Joules to kWh and therms in new dataframe
get_monthly95["FanEnergy(kBtu)"] = get_monthly95["FanEnergy[J](Hourly)"] * .00000094781712
get_monthly95["HeatingEnergy(kBtu)"] = get_monthly95["HeatingEnergy[J](Hourly)"] * .00000094781712
get_monthly95["CoolingEnergy(kBtu)"] = get_monthly95["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_monthly95

In [ ]:
# Drop columns
get_monthly95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly95

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [ ]:
# Copy dataframe to modify and leave original df intact
get_monthly95_costs = all_data95_2.copy(deep=True)

#get_monthly95

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly95_costs["FanCost($)"] = (get_monthly95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly95_costs["HeatingCost($)"] = ((get_monthly95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly95_costs["CoolingCost($)"] = (get_monthly95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly95_costs

In [ ]:
# Drop columns
get_monthly95_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly95_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [ ]:
# Combine data for energy and cost
allMonthlyData95 = pd.merge(get_monthly95, get_monthly95_costs, how="inner")

#allMonthlyData95

In [ ]:
# Add energy and cost totals

totalEnergyMonthly95 = allMonthlyData95["FanEnergy(kBtu)"] + allMonthlyData95["HeatingEnergy(kBtu)"] + allMonthlyData95["CoolingEnergy(kBtu)"]
totalCostMonthly95 = allMonthlyData95["FanCost($)"] + allMonthlyData95["HeatingCost($)"] + allMonthlyData95["CoolingCost($)"]
allMonthlyData95["MonthlyEnergy(kBtu)"] = totalEnergyMonthly95
allMonthlyData95["MonthlyCost($)"] = totalCostMonthly95

#allMonthlyData95

In [ ]:
allMonthlyData95["Month"] = allMonthlyData95.Date_Time.str.slice(0,3)

allMonthlyData95

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [ ]:
# Group by scenario and date/time for export to graphing
MonthlyData95_2 = allMonthlyData95.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData95_2.sum()

In [ ]:
MonthlyDataFinal95 = MonthlyData95_2.sum()

MonthlyDataFinal95

In [ ]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal95.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [ ]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal95.round(2).to_csv("../Regression_Scraper_Output/hou_monthly95_data.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [ ]:
# Copy dataframe to modify and leave original df intact
all_data95_3 = all_data95_1.copy(deep=True)

#all_data95_3

In [ ]:
# Get wanted columns
# Include hourly illumance
all_data95_3 = all_data95_3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data95_3

In [ ]:
# Rename Columns
get_hourly95 = all_data95_3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly95

In [ ]:
# Convert Joules to kBtus in new dataframe
get_hourly95["FanEnergy(kBtu)"] = get_hourly95["FanEnergy[J](Hourly)"] * .00000094781712
get_hourly95["HeatingEnergy(kBtu)"] = get_hourly95["HeatingEnergy[J](Hourly)"] * .00000094781712
get_hourly95["CoolingEnergy(kBtu)"] = get_hourly95["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_hourly95

In [ ]:
# Drop columns
get_hourly95.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly95

In [ ]:
#list(get_hourly95.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [ ]:
# Copy dataframe to modify and leave original df intact
get_hourly95_costs = all_data95_3.copy(deep=True)

#get_hourly95_costs

In [ ]:
# Rename Columns
get_hourly95_costs = get_hourly95_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly95_costs

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly95_costs["FanCost($)"] = (get_hourly95_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly95_costs["HeatingCost($)"] = ((get_hourly95_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly95_costs["CoolingCost($)"] = (get_hourly95_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

##get_hourly95_costs

In [ ]:
# Drop columns
get_hourly95_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly95_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [ ]:
# Combine data for energy and cost
allHourlyData95 = pd.merge(get_hourly95, get_hourly95_costs, how="inner")

#allHourlyData95

In [ ]:
#Extract hour, day, month.
allHourlyData95["Month"] = allHourlyData95.Date_Time.str.slice(0,3)
allHourlyData95["Day"] = allHourlyData95.Date_Time.str.slice(4,6)
allHourlyData95["Hour"] = allHourlyData95["Date_Time"].astype(str).str[8:16]

allHourlyData95

In [ ]:
# Add energy and cost totals

totalEnergyhourly95 = allHourlyData95["FanEnergy(kBtu)"] + allHourlyData95["HeatingEnergy(kBtu)"] + allHourlyData95["CoolingEnergy(kBtu)"]
totalCosthourly95 = allHourlyData95["FanCost($)"] + allHourlyData95["HeatingCost($)"] + allHourlyData95["CoolingCost($)"]
allHourlyData95["HourlyEnergy(kBtu)"] = totalEnergyhourly95
allHourlyData95["HourlyCost($)"] = totalCosthourly95

#allHourlyData95

In [ ]:
# Export grouped file to csv (used before grouped by month)
allHourlyData95.round(2).to_csv("../Regression_Scraper_Output/hou_hourly95_data.csv", header=True, index=False)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## ---

## Combine v9.2 CSV Files in Directory

In [ ]:
# Create variable for files in directory
files = [f for f in os.listdir("data/92/hou/") if f.endswith(".csv")]

# files

In [ ]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data92 = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/92/hou/" + file)
    all_data92 = pd.concat([all_data92, df])
    
all_data92

# 429240  rows, 214 columns

In [ ]:
# Get wanted columns
all_data92_1 = all_data92[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]

all_data92_1

In [ ]:
# all_data92_1 = all_data92_1.dropna().reset_index(drop=True)

# all_data92_1

# #438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [ ]:
# Rename Columns
all_data92_2 = all_data92_1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data92_2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [ ]:
# Copy dataframe to modify and leave original df intact
get_annual92 = all_data92_2.copy(deep=True)

#get_annual92

In [ ]:
# Convert Joules to KBtu in dataframe
get_annual92["FanEnergy(kBtu)"] = get_annual92["FanEnergy[J](Hourly)"] * .00000094781712
get_annual92["HeatingEnergy(kBtu)"] = get_annual92["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual92["CoolingEnergy(kBtu)"] = get_annual92["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual92

In [ ]:
#list(get_annual92.columns.values)

In [ ]:
# Drop columns
get_annual92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual92

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [ ]:
# Get annual costs
get_annual92_costs = all_data92_2.copy(deep=True)

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual92_costs["FanCost($)"] = (get_annual92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual92_costs["HeatingCost($)"] = ((get_annual92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual92_costs["CoolingCost($)"] = (get_annual92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual92_costs

In [ ]:
# Drop columns
get_annual92_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual92_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [ ]:
# Combine data for energy and cost
allAnnualData92 = pd.merge(get_annual92, get_annual92_costs)

#allAnnualData2

In [ ]:
# Add energy and cost totals
totalEnergy92 = allAnnualData92["FanEnergy(kBtu)"] + allAnnualData92["HeatingEnergy(kBtu)"] + allAnnualData92["CoolingEnergy(kBtu)"]
allAnnualData92["AnnualEnergy(kBtu)"] = totalEnergy92

totalCost92 = allAnnualData92["FanCost($)"] + allAnnualData92["HeatingCost($)"] + allAnnualData92["CoolingCost($)"]
allAnnualData92["AnnualCost($)"] = totalCost92

#allAnnualData92

In [ ]:
#list(allAnnualData92.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [ ]:
# Group by scenario
annualData92 = allAnnualData92.groupby(["Scenario"], as_index=True)

#annualData92.sum().round(2)

In [ ]:
annualDataFinal92 = annualData92.sum().round(2)

#annualDataFinal92

In [ ]:
# Export grouped file to csv
annualDataFinal92.to_csv("../Regression_Scraper_Output/hou_annual92_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh for costs only, convert to kbtu for data<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [ ]:
# Copy dataframe to modify and leave original df intact
get_monthly92 = all_data92_2.copy(deep=True)

get_monthly92

In [ ]:
# Convert Joules to kWh and therms in new dataframe
get_monthly92["FanEnergy(kBtu)"] = get_monthly92["FanEnergy[J](Hourly)"] * .00000094781712
get_monthly92["HeatingEnergy(kBtu)"] = get_monthly92["HeatingEnergy[J](Hourly)"] * .00000094781712
get_monthly92["CoolingEnergy(kBtu)"] = get_monthly92["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_monthly92

In [ ]:
# Drop columns
get_monthly92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly92

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [ ]:
# Copy dataframe to modify and leave original df intact
get_monthly92_costs = all_data92_2.copy(deep=True)

#get_monthly92

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly92_costs["FanCost($)"] = (get_monthly92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly92_costs["HeatingCost($)"] = ((get_monthly92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly92_costs["CoolingCost($)"] = (get_monthly92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly92_costs

In [ ]:
# Drop columns
get_monthly92_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly92_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [ ]:
# Combine data for energy and cost
allMonthlyData92 = pd.merge(get_monthly92, get_monthly92_costs, how="inner")

#allMonthlyData92

In [ ]:
# Add energy and cost totals

totalEnergyMonthly92 = allMonthlyData92["FanEnergy(kBtu)"] + allMonthlyData92["HeatingEnergy(kBtu)"] + allMonthlyData92["CoolingEnergy(kBtu)"]
totalCostMonthly92 = allMonthlyData92["FanCost($)"] + allMonthlyData92["HeatingCost($)"] + allMonthlyData92["CoolingCost($)"]
allMonthlyData92["MonthlyEnergy(kBtu)"] = totalEnergyMonthly92
allMonthlyData92["MonthlyCost($)"] = totalCostMonthly92

#allMonthlyData92

In [ ]:
allMonthlyData92["Month"] = allMonthlyData92.Date_Time.str.slice(0,3)

allMonthlyData92

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [ ]:
# Group by scenario and date/time for export to graphing
MonthlyData92_2 = allMonthlyData92.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData92_2.sum()

In [ ]:
MonthlyDataFinal92 = MonthlyData92_2.sum()

#MonthlyDataFinal92

In [ ]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal92.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [ ]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal92.round(2).to_csv("../Regression_Scraper_Output/hou_monthly92_data.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [ ]:
# Copy dataframe to modify and leave original df intact
all_data92_3 = all_data92_1.copy(deep=True)

#all_data92_3

In [ ]:
# Get wanted columns
# Include hourly illumance
all_data92_3 = all_data92_3[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) "]]
#all_data92_3

In [ ]:
# Rename Columns
get_hourly92 = all_data92_3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly92

In [ ]:
# Convert Joules to kBtus in new dataframe
get_hourly92["FanEnergy(kBtu)"] = get_hourly92["FanEnergy[J](Hourly)"] * .00000094781712
get_hourly92["HeatingEnergy(kBtu)"] = get_hourly92["HeatingEnergy[J](Hourly)"] * .00000094781712
get_hourly92["CoolingEnergy(kBtu)"] = get_hourly92["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_hourly92

In [ ]:
# Drop columns
get_hourly92.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly92

In [ ]:
#list(get_hourly92.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [ ]:
# Copy dataframe to modify and leave original df intact
get_hourly92_costs = all_data92_3.copy(deep=True)

#get_hourly92_costs

In [ ]:
# Rename Columns
get_hourly92_costs = get_hourly92_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electric Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil Gas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electric Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly92_costs

In [ ]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly92_costs["FanCost($)"] = (get_hourly92_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly92_costs["HeatingCost($)"] = ((get_hourly92_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly92_costs["CoolingCost($)"] = (get_hourly92_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

##get_hourly92_costs

In [ ]:
# Drop columns
get_hourly92_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly92_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Extract Date_Time<br>
Export combine file

In [ ]:
# Combine data for energy and cost
allHourlyData92 = pd.merge(get_hourly92, get_hourly92_costs, how="inner")

#allHourlyData92

In [ ]:
#Extract hour, day, month.
allHourlyData92["Month"] = allHourlyData92.Date_Time.str.slice(0,3)
allHourlyData92["Day"] = allHourlyData92.Date_Time.str.slice(4,6)
allHourlyData92["Hour"] = allHourlyData92["Date_Time"].astype(str).str[8:16]

allHourlyData92

In [ ]:
# Add energy and cost totals

totalEnergyhourly92 = allHourlyData92["FanEnergy(kBtu)"] + allHourlyData92["HeatingEnergy(kBtu)"] + allHourlyData92["CoolingEnergy(kBtu)"]
totalCosthourly92 = allHourlyData92["FanCost($)"] + allHourlyData92["HeatingCost($)"] + allHourlyData92["CoolingCost($)"]
allHourlyData92["HourlyEnergy(kBtu)"] = totalEnergyhourly92
allHourlyData92["HourlyCost($)"] = totalCosthourly92

#allHourlyData92

In [ ]:
# Export grouped file to csv (used before grouped by month)
allHourlyData92.round(2).to_csv("../Regression_Scraper_Output/hou_hourly92_data.csv", header=True, index=False)